# Womens Rugby Team Graph

A graph of who pairwise matches between teams. Visualising which teams frequently play each other. I also want to investigate the structure of the graph - do we detect sub-communities & if so, do they reflect features such as European or Southern tournaments.

Which teams are important - have high levels of centrality. How does this differ between 7s and 15s.

The initial plan is to use `networkx` package for visualisation, but to make it fancy I hope to utilise `pyvis`

In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

## 15s

I'll start with *proper rugby*, aka 15s rugby, I'll compare the results with 7s later on. 

### Data

First I load in the data, I need to generate the adjaceny matrix, this will result in a weighted graph where the edge weights correspond to the number of matches played between the pair of teams. Further, the vertex size will correspond to the win-rate of the team represented by that vertex.

In [6]:
df15s = pd.read_csv("./data/fifteens.csv")
df15s.head(5)

,test_no,date,team_1,score_1,score_2,team_2,venue,home_test_no,away_test_no,series_no,tournament,margin_of_victory,home_away_win,winner,loser
0,1,1982-06-13,Netherlands,0,4,France,Utrecht,1,1,1,Friendly,4,A,France,Netherlands
1,2,1983-06-05,France,10,0,Netherlands,La Teste,2,2,2,Friendly,10,H,France,Netherlands
2,3,1984-05-27,Netherlands,0,3,France,Hilversum,3,3,3,Friendly,3,A,France,Netherlands
3,4,1984-12-31,Sweden,0,34,Netherlands,Malmö,1,4,1,Friendly,34,A,Netherlands,Sweden
4,5,1985-06-02,France,20,0,Netherlands,Bourg en Bresse,4,5,4,Friendly,20,H,France,Netherlands


In [7]:
df15s.shape

(1468, 15)

### Adjacency matrix

Plan for contstructing $\hat{A}$.

1. Groupby team_1 and team_2.
2. Count number of matches.
3. Convert to wide format.
4. Convert to array/matrix and symmetrise by summation - this will handle the home/away challenge.